In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import keras
from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras.layers import Input, Conv2D,MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, concatenate, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import plot_model
from keras import layers, models, datasets,  regularizers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Dataset from https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset
train_dir = "/kaggle/input/face-expression-recognition-dataset/images/train/"
val_dir = "/kaggle/input/face-expression-recognition-dataset/images/validation/"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

In [ ]:
image_labels = {
    0 : "angry",
    1 : "disgust",
    2 : "fear",
    3 : "happy",
    4 : "neutral",
    5 : "sad",
    6 : "surprise"
}

In [ ]:
INPUT_SHAPE = (64, 64, 1)
KERNEL_SIZE = (3, 3)
model = models.Sequential()

model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))

In [ ]:
from keras.callbacks import LearningRateScheduler
import math

# Define a function to decay the learning rate
def lr_schedule(epoch):
    initial_lr = 0.001
    decay = 0.10
    epochs_drop = 10
    lr = initial_lr * math.pow(decay, math.floor((1+epoch)/epochs_drop))
    return lr

# Define the learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=val_generator,
                    callbacks = [lr_scheduler]
                    )

In [ ]:
model.evaluate(val_generator)

In [ ]:
def plot_results(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize = (24, 6))
    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')
    
    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'b', label = 'Training Loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
plot_results(history)

In [ ]:
images, labels = next(val_generator)
def display_grid(images, labels, model, image_labels):
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.flatten()

    for img, ax, lbl in zip(images[:16], axes, labels[:16]):
        ax.imshow(img.reshape(64, 64), cmap='gray')
        pred = model.predict(img.reshape((1, 64, 64, 1)))
        pred_label = image_labels[np.argmax(pred)]
        true_label = image_labels[np.argmax(lbl)]
        ax.set_title(f'Pred: {pred_label}\nTrue: {true_label}')
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
display_grid(images, labels, model, image_labels)

In [ ]:
model.save("models\emotion-detector.h5")